In [21]:
import torch
import torch.nn as nn

class Conv1d:
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=1):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding

        # Initialize the weights and biases
        self.weights = torch.randn(out_channels, in_channels, kernel_size)
        self.biases = torch.zeros(out_channels)

    def forward(self, x):
        batch_size, in_channels, width = x.shape

        # Calculate the output dimensions
        out_width = (width - self.kernel_size + 2 * self.padding) // self.stride + 1

        # Create an empty tensor for the output
        out = torch.zeros(batch_size, self.out_channels, out_width)

        # Apply the convolution operation
        for b in range(batch_size):
            for c_out in range(self.out_channels):
                for w_out in range(out_width):
                    w_start = w_out * self.stride - self.padding
                    w_end = w_start + self.kernel_size

                    receptive_field = x[b, :, w_start:w_end]
                    out[b, c_out, w_out] = torch.sum(receptive_field * self.weights[c_out]) + self.biases[c_out]

        return out
    
class CNN_Network(nn.Module):
    def __init__(self,input_size, sequence_length, output_size):
        super(CNN_Network, self).__init__()
        self.conv1 = Conv1d(in_channels=input_size, out_channels=32, kernel_size=2)
        self.conv2 = Conv1d(in_channels=32, out_channels=16, kernel_size=2)
        self.fc1 = nn.Linear(16 * 4, 84)
        self.fc3 = nn.Linear(84, output_size)

    def forward(self, x):
        x = torch.relu(self.conv1.forward(x))
        x = torch.relu(self.conv2.forward(x))
        x = x.view(-1, 16 * 4)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [22]:
from config import *
from load_data import getTrainingSet


dataset = getTrainingSet(reshape=False, sequence_length=24)
features = len(dataset.X[0][0])
sequence_length = len(dataset.X[0])
input_size = features
output_size = OUTPUT_SIZE
epochs = EPOCHS_FNN
learning_rate = LEARNING_RATE
batch_size = BATCH_SIZE

model = CNN_Network(input_size, sequence_length, output_size)
loss_function = nn.MSELoss()
adam = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [23]:
def train(model, dataset, loss_function, optimizer, batch_size, epochs):
    # Set the model to training mode
    model.train()

    # Create data loader
    data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # Training loop
    for epoch in range(epochs):
        running_loss = 0.0

        # Iterate over the data loader
        for inputs, labels in data_loader:
            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)

            # Calculate the loss
            loss = loss_function(outputs, labels)

            # Backward pass
            loss.backward()

            # Update the weights
            optimizer.step()

            # Update the running loss
            running_loss += loss.item()

        # Print the average loss for the epoch
        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(data_loader)}")

    print("Training complete.")


In [24]:
train(model, dataset, loss_function, adam, batch_size, epochs)

RuntimeError: The size of tensor a (0) must match the size of tensor b (2) at non-singleton dimension 1